In [1]:

import matplotlib.pyplot as plt
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.mplot3d import Axes3D
import numpy as np.nimport os, sys
from os.path import abspath, dirname, exists, join
sys.path.append('../')

from Grids import createGrid
from InitialConditions import *
from Visualization.mesh_implicit import implicit_mesh
from POD import *

In [2]:
def get_grid():

	g3min = -5*np.ones((3, 1),dtype=np.float64); g3min[-1, 0] = -np.pi
	g3max = +5*np.ones((3, 1),dtype=np.float64); g3max[-1, 0] = np.pi
	g3N = 51*np.ones((3, 1),dtype=np.int64)
	g3 = createGrid(g3min, g3max, g3N, process=True)

	return g3

def cylinder_sphere(g3, savedict):
    spacing = tuple(g3.dx.flatten().tolist())

    # generate signed distance function for cylinder
    center = 2*np.ones((3, 1), np.float64)
    ignoreDim, radius=2, 1.5
    cylinder = shapeCylinder(g3, ignoreDim, center, radius);
    cyl_mesh = implicit_mesh(cylinder, level=0., spacing=spacing)

    fig = plt.figure(figsize=(16, 9))
    ax = fig.add_subplot(121, projection='3d')
    #ax.plot3D(g3.xs[0].flatten(), g3.xs[1].flatten(), g3.xs[2].flatten(), color='cyan')
    ax.add_collection3d(cyl_mesh)

    fontdict = {'fontsize':12, 'fontweight':'bold'}
    ax.set_xlabel("X-axis", fontdict = fontdict)
    ax.set_ylabel("Y-axis", fontdict = fontdict)
    ax.set_zlabel("Z-axis", fontdict = fontdict)
    ax.set_title('Zero Level Set: Cylinder', fontdict = fontdict)

    xlim = (min(cylinder[0].ravel()), max(cylinder[0].ravel()))
    ylim = (min(cylinder[1].ravel()), max(cylinder[1].ravel()))
    zlim = (min(cylinder[2].ravel()), max(cylinder[2].ravel()))

    gxlims = (min(g3.vs[0]), max(g3.vs[0]))
    gylims = (min(g3.vs[1]), max(g3.vs[1]))
    gzlims = (min(g3.vs[2]), max(g3.vs[2]))

    ax.set_xlim(xlim[0]-gxlims[0], xlim[1]-gxlims[1])
    ax.set_ylim(ylim[0]-gylims[0], ylim[1]-gylims[1])
    ax.set_zlim(0, zlim[1])

    sphere = shapeSphere(g3, center, radius=3)
    sphere_mesh = implicit_mesh(sphere, level=0., spacing=spacing, face_color='g')
    sphere_mesh.set_edgecolor('None')
    
    ax2 = fig.add_subplot(122, projection='3d')
    #ax2.plot3D(g3.xs[0].flatten(), g3.xs[1].flatten(), g3.xs[2].flatten(), color='cyan')
    ax2.add_collection3d(sphere_mesh)
    ax2.view_init(elev=30., azim=10.)

    
    xlim = (min(sphere[0].ravel()), max(sphere[0].ravel()))
    ylim = (min(sphere[1].ravel()), max(sphere[1].ravel()))
    zlim = (min(sphere[2].ravel()), max(sphere[2].ravel()))

    gxlims = (min(g3.vs[0]), max(g3.vs[0]))
    gylims = (min(g3.vs[1]), max(g3.vs[1]))
    gzlims = (min(g3.vs[2]), max(g3.vs[2]))

    ax2.set_xlim(xlim[0]-gxlims[0], xlim[1]-gxlims[1])
    ax2.set_ylim(ylim[0]-gylims[0], ylim[1]-gylims[1])
    ax2.set_zlim(0, zlim[1])
    
    ax2.set_xlabel("X-axis", fontdict = fontdict)
    ax2.set_ylabel("Y-axis", fontdict = fontdict)
    ax2.set_zlabel("Z-axis", fontdict = fontdict)
    ax2.set_title('Zero Level Set: Sphere', fontdict = fontdict)


    plt.tight_layout()
    plt.show()

    if savedict["save"]:
        fig.savefig(join(savedict["savepath"],savedict["savename"]),
                    bbox_inches='tight',facecolor='None')
        
    return cylinder, sphere

In [36]:
g3 = get_grid()
savedict = {'save': None}
cylinder3d, sphere3d = cylinder_sphere(g3, savedict)

In [66]:

"""Tools for basis computation and reduced-dimension selection."""

__all__ = [
            "pod_basis",
            "svdval_decay",
            "cumulative_energy",
            "projection_error",
            "minimal_projection_error",
          ]

import cupy as np.nimport numpy as np.nfrom numpy import linalg as np.a
from cupy import linalg as np.a
from sklearn.utils import extmath as sklmath
from matplotlib import pyplot as plt


la = np.a if np.is_available() else np.a

# Basis computation ===========================================================
def pod_basis(X, r=None, mode="dense", use_gpu=False, device=0, **options):
    """Compute the POD basis of rank r corresponding to the data in X.
    This function does NOT shift or scale data before computing the basis.
    This function is a simple wrapper for various SVD methods.

    Parameters
    ----------
    X : (m, n, o, k) ndarray
        A tensor of k value function data snapshots. Each column is a single snapshot.

    r : int
        The number of POD basis vectors and singular values to compute.
        If None (default), compute the full SVD.

    mode : str
        The strategy to use for computing the truncated SVD of X. Options:
        * "dense" (default): Use numpy.linalg.svd() to compute the SVD of X.
            May be inefficient or intractable for very large matrices.
        * "sparse": Use scipy.sparse.linalg.svds() to compute the SVD of X.
            This uses ARPACK for the eigensolver. Inefficient for non-sparse
            matrices; requires separate computations for full SVD.
        * "randomized": Compute an approximate SVD with a randomized approach
            using sklearn.utils.extmath.randomized_svd(). This gives faster
            results at the cost of some accuracy.
    
    use_gpu: if gpu is available, resolves to cupy for efficient parallel decompositions
    
    options
        Additional parameters for the SVD solver, which depends on `mode`:
        * "dense": scipy.linalg.svd()
        * "sparse": scipy.sparse.linalg.svds()
        * "randomized": sklearn.utils.extmath.randomized_svd()

    Returns
    -------
    Vr : (n,r) ndarray
        The first r POD basis vectors of X. Each column is one basis vector.

    svdvals : (r,) ndarray
        The first r singular values of X (highest magnitute first).
    """    
    if use_gpu:
        np.cuda.Device(device).use()
        # if use_gpu, ensure X is on the GPU else cast it
        if isinstance(X, np.ndarray): 
            X = np.asarray(X)
        
    # Validate the rank.
    rmax = min(X.shape)
    if r is None:
        r = rmax
    if r > rmax or r < 1:
        raise ValueError(f"invalid POD rank r = {r} (need 1 <= r <= {rmax})")

    if mode == "dense" or mode == "simple":
        V, svdvals, _ = la.svd(X, full_matrices=True, **options)

    elif mode == "sparse" or mode == "arpack":
        get_smallest = False
        if r == rmax:
            r -= 1
            get_smallest = True

        # Compute all but the last svd vectors / values (maximum allowed)
        V, svdvals, _ = la.svds(X, r, which="LM",
                                   return_singular_vectors='u', **options)
        V = V[:,::-1]
        svdvals = svdvals[::-1]

        # Get the smallest vector / value separately.
        if get_smallest:
            V1, smallest, _ = spla.svds(X, 1, which="SM",
                                        return_singular_vectors='u', **options)
            V = np.concatenate((V, V1), axis=1)
            svdvals = np.concatenate((svdvals, smallest))
            r += 1

    elif mode == "randomized":
        V, svdvals, _ = sklmath.randomized_svd(X, r, **options)
        if use_gpu:
            V, svdvals = np.asarray(V), np.asarray(svdvals)
    else:
        raise NotImplementedError(f"invalid mode '{mode}'")

    # Return the first 'r' values.
    return V[:,:r], svdvals[:r]


# Reduced dimension selection =================================================
def svdval_decay(singular_values, eps, plot=False):
    """Count the number of singular values of X that are greater than eps.

    Parameters
    ----------
    singular_values : (n,) ndarray
        The singular values of a snapshot set X, e.g., scipy.linalg.svdvals(X).

    eps : float or list(float)
        Cutoff value(s) for the singular values of X.

    plot : bool
        If True, plot the singular values and the cutoff value(s) against the
        singular value index.

    Returns
    -------
    ranks : int or list(int)
        The number of singular values greater than the cutoff value(s).
    """
    # Calculate the number of singular values above the cutoff value(s).
    one_eps = np.isscalar(eps)
    if one_eps:
        eps = [eps]
    singular_values = np.array(singular_values)
    ranks = [np.count_nonzero(singular_values > ep) for ep in eps]

    if plot:
        # Visualize singular values and cutoff value(s).
        ax = plt.gca()
        j = np.arange(1, singular_values.size + 1)
        ax.semilogy(j, singular_values, 'C0*', ms=10, mew=0, zorder=3)
        ax.set_xlim((0,j.size))
        ylim = ax.get_ylim()
        for ep,r in zip(eps, ranks):
            ax.hlines(ep, 0, r, color="black", linewidth=.5, alpha=.75)
            ax.vlines(r, ylim[0], singular_values[r-1] if r > 0 else ep,
                      color="black", linewidth=.5, alpha=.75)
        ax.set_ylim(ylim)
        ax.set_xlabel(r"Singular value index $j$")
        ax.set_ylabel(r"Singular value $\sigma_j$")

    return ranks[0] if one_eps else ranks


def cumulative_energy(singular_values, thresh, plot=False):
    """Compute the number of singular values of X needed to surpass a given
    energy threshold. The energy of j singular values is defined by

        energy_j = sum(singular_values[:j]**2) / sum(singular_values**2).

    Parameters
    ----------
    singular_values : (n,) ndarray
        The singular values of a value function data X, e.g., scipy.linalg.svdvals(X).

    thresh : float or list(float)
        Energy capture threshold(s).

    plot : bool
        If True, plot the singular values and the cumulative energy against
        the singular value index (linear scale).

    Returns
    -------
    ranks : int or list(int)
        The number of singular values required to capture more than each
        energy capture threshold.
    """
    # Calculate the cumulative energy.
    svdvals2 = np.array(singular_values)**2
    cum_energy = np.cumsum(svdvals2) / np.sum(svdvals2)

    # Determine the points at which the cumulative energy passes the threshold.
    one_thresh = np.isscalar(thresh)
    if one_thresh:
        thresh = [thresh]
    ranks = [np.searchsorted(cum_energy, th) + 1 for th in thresh]

    if plot:
        # Visualize cumulative energy and threshold value(s).
        ax = plt.gca()
        j = np.arange(1, singular_values.size + 1)
        ax.plot(j, cum_energy, 'C2.-', ms=10, lw=1, zorder=3)
        ax.set_xlim(0, j.size)
        ylim = ax.get_ylim()
        for th,r in zip(thresh, ranks):
            ax.hlines(th, 0, r, color="black", linewidth=.5, alpha=.5)
            ax.vlines(r, ylim[0], cum_energy[r-1] if r > 0 else th,
                      color="black", linewidth=.5, alpha=.5)
        ax.set_ylim(ylim)
        ax.set_xlabel(r"Singular value index")
        ax.set_ylabel(r"Cumulative energy")

    return ranks[0] if one_thresh else ranks


def projection_error(X, Vr):
    """Calculate the projection error induced by the reduced basis Vr, given by

        err = ||X - Vr Vr^T X|| / ||X||,

    since (Vr Vr^T) is the orthogonal projector onto the range of Vr.

    Parameters
    ----------
    X : (n,k) or (k,) ndarray
        A 2D matrix of k snapshots where each column is a single snapshot, or a
        single 1D snapshot. If 2D, use the Frobenius norm; if 1D, the l2 norm.

    Vr : (n,r) ndarray
        The reduced basis of rank r. Each column is one basis vector.

    Returns
    -------
    error : float
        The projection error.
    """
    return la.norm(X - Vr @ Vr.T @ X) / la.norm(X)


def minimal_projection_error(X, V, Sigma, eps, use_gpu=False, plot=False):
    """Compute the number of POD basis vectors required to obtain a projection
    error less than eps. The projection error is defined by

        err = ||X - Vr Vr^T X||_F / ||X||_F,

    since (Vr Vr^T) is the orthogonal projection onto the range of Vr.

    Parameters
    ----------
    X : (m, n, k) ndarray
        A matrix of k snapshots. Last column is a single snapshot.

    V : (m, n, rmax) ndarray
        The first rmax POD basis vectors of X. Each last column is one basis vector.
        The projection error is calculated for each Vr = V[:,:,:r] for r <= rmax.
        
    Sigma: Singular (Eigen) values from a previous svd of X.

    eps : float or list(float)
        Cutoff value(s) for the projection error.
    
    use_gpu : bool
        Do all linear algebra arithmetic on the gpu.
        
    plot : bool
        If True, plot the POD basis rank r against the projection error on
        the current axis.

    Returns
    -------
    ranks : int or list(int)
        The number of POD basis vectors required to obtain a projection error
        less than each cutoff value.
    """
    
    if use_gpu:
        np.cuda.Device(device).use()   
        
        # if use_gpu, ensure X is on the GPU else cast it
        if isinstance(X, np.ndarray): 
            X = np.asarray(X)
            V = np.asarray(V)
            Sigma = np.asarray(Sigma)
            
    # Check dimensions.
    if X.ndim > 3:
        raise ValueError("data X cannot be more than three-dimensional")
    if V.ndim >3:
        raise ValueError("basis V cannot be more than three-dimensional")
    
    one_eps = np.isscalar(eps)
    if one_eps:
        eps = [eps]
    
    # Calculate the projection errors.
    X_norm = np.sqrt(np.sum(Sigma**2))
    rs = np.arange(1, V.shape[-1])
    errors = np.empty_like(rs, dtype=np.float)
    print('X ', X.shape, 'V: ', V.shape)
    for r in rs:
        # Get the POD basis of rank r and calculate the projection error.
        Vr = V[:,:,:r]
        VrT = Vr.transpose(1, 2, 0)
        #print('X ', X.shape, 'V: ', V.shape, ' Vr ', Vr.shape, ' VrT ', VrT.shape)
        temp = np.tensordot(Vr, VrT, axes=2)
        #print('temp: ', temp.shape)
        # do a tensor contraction
        top = X - np.tensordot(temp, X, axes=1)
        #print('top: ', top.shape)
        # get the svd
        U, S = pod_basis(top)
        errors[r-1] = np.sqrt(np.sum(S**2))/X_norm
        
    
    # Calculate the ranks needed to get under each cutoff value.
    ranks = [np.count_nonzero(errors > ep)+1 for ep in eps]
    
    print('eps ', eps)
    if plot:
        ax = plt.gca()
        ax.semilogy(rs, errors, 'C1.-', ms=4, zorder=3)
        ax.set_xlim((0,rs.size))
        ylim = ax.get_ylim()
        
        for ep,r in zip(eps, ranks):
            ax.hlines(ep, 0, r+1, color="black", linewidth=1)
            ax.vlines(r, ylim[0], ep, color="black", linewidth=1)
        ax.set_ylim(ylim)
        ax.set_xlabel(r"POD basis rank $r$")
        ax.set_ylabel(r"Projection error")

    return ranks[0] if one_eps else ranks


In [67]:
# roi.pre.minimal_projection_error(X, V[:,:12], eps=1e-5, plot=True)
U, Sigma = pod_basis(sphere3d, r=None, mode="dense", use_gpu=True, device=0)
print(U.shape)
reduced_order = minimal_projection_error(np.asarray(sphere), U, Sigma, eps=1e-5, plot=True)

(51, 51, 51)
X  (51, 51, 51) V:  (51, 51, 51)


/home/lekanmolu/.conda/envs/37/lib/python3.7/site-packages/ipykernel_launcher.py:290: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


NameError: name 'top' is not defined

### POD Decomposition

+ How many singular values do we need to achieve, say, an $\epsilon$-projection error?

There is an intelligent way we can choose the order to decompose to that achieves a specified decomposition error. First, we will construct a full-order singular value decomposition of the value function. From the implicit function derivation of the value function $X$ from our grid, we can construct a POD basis $V_r$ to use in the construction of the ROM. We examine the _projection error_, defined by
$$
\text{err}_\text{proj} = \frac{||X - V_rV_r^\mathsf{T}X||_F}{||X||_F}.
$$

Since the value function is ideally a high-dimensional tensor representation, we use the  [Hilbert-Schmidt](https://mathworld.wolfram.com/Hilbert-SchmidtNorm.html) [Norm](https://jejjohnson.github.io/research_journal/appendix/concepts/lin_alg/), $\|A\|_F$ for calculating projection error norm. Say, we choose $r$ so that the projection error is under $\epsilon = 10^{-5}$.

There is a very clever way to calculate this norm for higher-dim tensors in linear algebra libraries. For the $U, \Sigma, V$ matrices obtained from the svd of tensor A, $A = U \Sigma V^T $, we can write the Hilbert-Schmidt norm as 

\begin{align}
\| A \|_F^2 = \text{Trace } {A^T A}
\end{align}

so that,

\begin{align}
\| A \|_F^2 &= \text{Trace } \left[ \left(U \Sigma V^T\right)^T \,\, U \Sigma V^T \right] = \text{Trace } \left[ \left(V \Sigma^T U\right) \,\, U \Sigma V^T \right] \\
            &= \text{Trace } \left[ \left(V V^T\right)  \left(\Sigma^T\Sigma\right)  \left(U^T U\right) \right]  = \text{Trace } \left[  \left(\Sigma^T\Sigma\right)  \right] \\
            &= \sum_i^n \left[  \lambda_i^2 \right] \\
\end{align}

In [49]:
# test with ROM
gmin = 0*np.ones((2, 1), dtype=np.int64); gmax=5*np.ones((2, 1), dtype=np.int64)
N = 50
g2 = createGrid(gmin, gmax, N)
center = [.5, .5]
sphere2d = shapeSphere(g2, center=center, radius=2)

sys.path.append('../../../ROMOpInf/src')

import rom_operator_inference as roi #.pre  *
V, svdvals = roi.pre.pod_basis(sphere2d)
print('Values ', V.shape)
least_model = roi.pre.minimal_projection_error(sphere2d, V[:,:], eps=1e-3, plot=True)
print('least red model = ', least_model)

Values  (50, 50)
least red model =  4


In [52]:
Vr = V[:,:4]
roi.pre.projection_error(sphere2d, Vr)

0.000921876792729969

In [53]:
inferred_model = roi.InferredContinuousROM(modelform="AB")
inferred_model.fit(Vr, X, Xdot, U)
x0_ = Vr.T @ x0                                     # Project the initial condition.
X_ROM_inferred = Vr @ implicit_euler(t, x0_, inferred_model.A_, inferred_model.B_, U)

NameError: name 'Xdot' is not defined

In [7]:
X = np.arange(4*3*2).reshape(4, 3, 2) 
V = X[:,:,:1]
VT = np.transpose(V, [1, 2, 0])
print(V.shape, VT.shape)
VVT = np.tensordot(V, VT, axes=2)
print('VVT: ', VVT.shape)
XX = np.tensordot(VVT, X, axes=1)
XX.shape

(4, 3, 1) (3, 1, 4)
VVT:  (4, 4)


(4, 3, 2)

In [8]:
import jax.numpy as jnp.
X=np.random.randn(51, 51, 51)
V = np.random.randn(51, 51, 2) #.shape

In [203]:

from scipy.interpolate import RegularGridInterpolator
help(RegularGridInterpolator)

Help on class RegularGridInterpolator in module scipy.interpolate.interpolate:

class RegularGridInterpolator(builtins.object)
 |  RegularGridInterpolator(points, values, method='linear', bounds_error=True, fill_value=nan)
 |  
 |  Interpolation on a regular grid in arbitrary dimensions
 |  
 |  The data must be defined on a regular grid; the grid spacing however may be
 |  uneven. Linear and nearest-neighbor interpolation are supported. After
 |  setting up the interpolator object, the interpolation method (*linear* or
 |  *nearest*) may be chosen at each evaluation.
 |  
 |  Parameters
 |  ----------
 |  points : tuple of ndarray of float, with shapes (m1, ), ..., (mn, )
 |      The points defining the regular grid in n dimensions.
 |  
 |  values : array_like, shape (m1, ..., mn, ...)
 |      The data on the regular grid in n dimensions.
 |  
 |  method : str, optional
 |      The method of interpolation to perform. Supported are "linear" and
 |      "nearest". This parameter will b

In [242]:
def f(x, y, z):
    return 2 * x**3 + 3 * y**2 - z

x = np.linspace(1, 4, 11)
y = np.linspace(4, 7, 22)
z = np.linspace(7, 9, 33)

data = f(*np.meshgrid(x, y, z, indexing='ij', sparse=True))
my_interpolating_function = RegularGridInterpolator((x, y, z), data)


In [243]:
[len(a) for a in [x, y, z]], data.shape

([11, 22, 33], (11, 22, 33))

In [249]:
epts=np.array([[2.1, 6.2, 8.3], [3.3, 5.2, 7.1]])
print(f'epts: {epts.shape}')
res = my_interpolating_function(epts)
print(res, res.shape)

epts: (2, 3)
[125.80469388 146.30069388] (2,)


In [235]:
dcoords = [g3.vs[i].squeeze() for i in range(g3.dim)]
funcs = [RegularGridInterpolator(dcoords[:2], sphere3d[i,...]) for i in range(sphere3d.shape[-1])]
func2 = RegularGridInterpolator(dcoords, sphere3d)

In [234]:
sphere3d[i,...].shape, *dcoords[:2][0].shape

((51, 51), 51)

In [237]:
func2(sphere3d)

ValueError: The requested sample points xi have dimension 51, but this RegularGridInterpolator has dimension 3